# Random Forest

In [1]:
# Tratamiento de datos
import numpy as np
import pandas as pd
import sidetable as stb

# Gráficos
import matplotlib.pyplot as plt
import seaborn as sns

# Modelado y evaluación
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn import tree
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import GridSearchCV

# Barra de progreso de un proceso
from tqdm import tqdm

# Configuración warnings
import warnings
warnings.filterwarnings('once')

import sys
sys.path.append("../../")
from src import funciones as fun
from src import variables as var

En el pair programming de hoy debéis usar el csv que guardastéis cuando hicistéis el pairprgramming de codificicación (este csv debería tener las variables estadandarizas).

Objetivo:
- Ajustar el modelo a un Random Forest



Orden

In [71]:
df_ord = pd.read_pickle('../archivos/country_pesos.pkl')
df_ord.drop('basic_boxcox', axis=1, inplace=True)

In [74]:
df_ord.sample()

,mcdonalds,cappuccino,milk,rice,eggs,chicken,beef,banana,water,wine,...,gasoline,basic,internet,gym_monthly,cinema,preschool,primary_school,apt_3beds_outcentre,monthly_salary,country_pesos
187,-0.350427,0.527778,2.916667,-0.369942,-0.181159,-0.307273,0.50538,0.341637,-0.065934,0.44281,...,-0.163934,-0.774522,-0.663079,-0.297359,-0.361648,0.131662,1.530402,-0.028541,-0.178539,0


In [76]:
Xor = df_ord.drop("basic", axis = 1)
yor = df_ord["basic"]

In [77]:
xor_train, xor_test, yor_train, yor_test = train_test_split(Xor, yor, test_size = 0.2, random_state = 42)

In [78]:
bosqueor = RandomForestRegressor(random_state =0)
bosqueor

RandomForestRegressor(random_state=0)

In [79]:
bosqueor.fit(xor_train, yor_train)

RandomForestRegressor(random_state=0)

In [80]:
yor_test_rf = bosqueor.predict(xor_test)
yor_train_rf = bosqueor.predict(xor_train)

In [81]:
rfor_results = fun.metricas(yor_test, yor_train, yor_test_rf, yor_train_rf, "Random Forest Orden 1")
rfor_results

,MAE,MSE,RMSE,R2,set,modelo
0,0.306854,0.202031,0.449478,0.592026,test,Random Forest Orden 1
1,0.111703,0.031069,0.176263,0.942993,train,Random Forest Orden 1


In [82]:
max_featuresor = np.sqrt(len(xor_train.columns))
max_featuresor

4.795831523312719

In [85]:
paramor1 = {"max_depth": [10, 12, 14, 16], 
        "max_features": [2, 3,4, 5],
        "min_samples_split": [10, 50, 100],
        "min_samples_leaf": [10,50,100]} 

In [86]:
gs_rfor = GridSearchCV(
            estimator=RandomForestRegressor(),
            param_grid= paramor1, 
            cv=10, 
            verbose=-1, 
            return_train_score = True, 
            scoring="neg_mean_squared_error") 

In [87]:
gs_rfor.fit(xor_train, yor_train)

GridSearchCV(cv=10, estimator=RandomForestRegressor(),
             param_grid={'max_depth': [10, 12, 14, 16],
                         'max_features': [2, 3, 4, 5],
                         'min_samples_leaf': [10, 50, 100],
                         'min_samples_split': [10, 50, 100]},
             return_train_score=True, scoring='neg_mean_squared_error',
             verbose=-1)

In [88]:
mejor_or1 = gs_rfor.best_estimator_
mejor_or1

RandomForestRegressor(max_depth=14, max_features=5, min_samples_leaf=10,
                      min_samples_split=10)

In [89]:
yor_test2_rf = mejor_or1.predict(xor_test)
yor_train2_rf = mejor_or1.predict(xor_train)

In [90]:
rfor2_results = fun.metricas(yor_test, yor_train, yor_test2_rf, yor_train2_rf, "Random Forest Orden 2")
rfor2_results

,MAE,MSE,RMSE,R2,set,modelo
0,0.317902,0.215597,0.464325,0.564630,test,Random Forest Orden 2
1,0.258829,0.169795,0.412062,0.688446,train,Random Forest Orden 2


In [2]:
df = pd.read_pickle('../archivos/country_dummie.pkl')
df.head()

,mcdonalds,cappuccino,milk,rice,eggs,chicken,beef,banana,water,wine,...,country_United Kingdom,country_United States,country_Uruguay,country_Uzbekistan,country_Vanuatu,country_Venezuela,country_Vietnam,country_Yemen,country_Zambia,country_Zimbabwe
0,-0.219373,0.601852,2.395833,1.046243,1.304348,0.747273,4.294342,3.245552,0.307692,1.218954,...,0,0,0,0,0,0,0,0,0,0
1,-0.350427,0.625000,3.520833,-0.289017,-0.014493,-0.292727,0.338771,0.241993,-0.142857,1.035948,...,0,0,0,0,0,0,0,0,0,0
2,-0.552707,0.421296,1.791667,-0.398844,-0.384058,-0.490909,0.148560,0.014235,-0.285714,0.570261,...,0,0,0,0,0,0,0,0,0,0
3,-0.923077,-0.069444,-0.625000,-0.514451,-0.934783,-0.505455,-0.656716,-1.081851,-0.461538,0.315359,...,0,0,0,0,0,0,0,0,0,0
4,-0.746439,-0.398148,-0.666667,-0.502890,-0.884058,-0.483636,-0.690038,-0.967972,-0.450549,0.315359,...,0,0,0,0,0,0,0,0,0,0


In [3]:
# Separamos el dataframe entre variables predictoras (X) y variable dependiente (y)
X = df.drop(["basic", "basic_boxcox"], axis = 1)
y = df["basic"]

In [4]:
# dividimos ambos datos en 'train' y 'test' en una proporción 80-20:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [5]:
# primero generamos un modelo sin especificar hiperparámetros para después ir ajustando
bosque = RandomForestRegressor(random_state =0)
bosque

RandomForestRegressor(random_state=0)

In [6]:
# lo entrenamos
bosque.fit(x_train, y_train)

RandomForestRegressor(random_state=0)

In [7]:
y_test_rf = bosque.predict(x_test)
y_train_rf = bosque.predict(x_train)

- Extraer las métricas

In [8]:
rf1_results = fun.metricas(y_test, y_train, y_test_rf, y_train_rf, "Random Forest 1")
rf1_results

,MAE,MSE,RMSE,R2,set,modelo
0,0.295399,0.193418,0.439793,0.609418,test,Random Forest 1
1,0.108429,0.029441,0.171584,0.945979,train,Random Forest 1


*Comprobamos que nuestro modelo tiene un problema de **underfitting**, dado que todos los valores de las métricas son más altas en el set de test que en el de train, excepto en **R2**, que es significativamente más baja, lo cual indica que no está realizando unas predicciones ajustadas a los datos reales.*

- Probamos a mejorar nuestro modelo cambiado los hiperparámetros

In [9]:
max_features = np.sqrt(len(x_train.columns))
max_features

15.198684153570664

In [10]:
param1 = {"max_depth": [10, 12, 14], 
        "max_features": [1,2,3,4],
        "min_samples_split": [10, 50, 100],
        "min_samples_leaf": [10,50,100]} 

In [11]:
gs_rf = GridSearchCV(
            estimator=RandomForestRegressor(),
            param_grid= param1, 
            cv=10, 
            verbose=-1, 
            return_train_score = True, 
            scoring="neg_mean_squared_error") 

In [12]:
gs_rf.fit(x_train, y_train)

GridSearchCV(cv=10, estimator=RandomForestRegressor(),
             param_grid={'max_depth': [10, 12, 14],
                         'max_features': [1, 2, 3, 4],
                         'min_samples_leaf': [10, 50, 100],
                         'min_samples_split': [10, 50, 100]},
             return_train_score=True, scoring='neg_mean_squared_error',
             verbose=-1)

In [13]:
bosque2 = gs_rf.best_estimator_
bosque2

RandomForestRegressor(max_depth=14, max_features=4, min_samples_leaf=10,
                      min_samples_split=10)

In [14]:
y_test2_rf = bosque2.predict(x_test)
y_train2_rf = bosque2.predict(x_train)

In [15]:
rf2_results = fun.metricas(y_test, y_train, y_test2_rf, y_train2_rf, "Random Forest 2")
rf2_results

,MAE,MSE,RMSE,R2,set,modelo
0,0.457024,0.355859,0.596539,0.281391,test,Random Forest 2
1,0.464539,0.397166,0.630211,0.271249,train,Random Forest 2


*Han salido una smetricas peores, lo que nos lleva a pensar que hay que aumentar los parámetros*

In [16]:
param2 = {"max_depth": [6,8,10,12], 
        "max_features": [3,4,5,6],
        "min_samples_split": [50, 100, 150],
        "min_samples_leaf": [5,10,20]} 

In [17]:
gs_rf2 = GridSearchCV(
            estimator=RandomForestRegressor(),
            param_grid= param2, 
            cv=10, 
            verbose=-1, 
            return_train_score = True, 
            scoring="neg_mean_squared_error") 

In [18]:
gs_rf2.fit(x_train, y_train)

GridSearchCV(cv=10, estimator=RandomForestRegressor(),
             param_grid={'max_depth': [6, 8, 10, 12],
                         'max_features': [3, 4, 5, 6],
                         'min_samples_leaf': [5, 10, 20],
                         'min_samples_split': [50, 100, 150]},
             return_train_score=True, scoring='neg_mean_squared_error',
             verbose=-1)

In [19]:
bosque3 = gs_rf2.best_estimator_
bosque3

RandomForestRegressor(max_depth=10, max_features=6, min_samples_leaf=5,
                      min_samples_split=50)

In [20]:
y_test3_rf = bosque3.predict(x_test)
y_train3_rf = bosque3.predict(x_train)

In [21]:
rf3_results = fun.metricas(y_test, y_train, y_test3_rf, y_train3_rf, "Random Forest 3")
rf3_results

,MAE,MSE,RMSE,R2,set,modelo
0,0.384101,0.273459,0.522933,0.447787,test,Random Forest 3
1,0.384721,0.298979,0.546790,0.451409,train,Random Forest 3


In [22]:
param3 = {"max_depth": [10, 12, 14], 
        "max_features": [12,13,14,15],
        "min_samples_split": [10, 50, 100],
        "min_samples_leaf": [10,50,100]} 

In [23]:
gs_rf3 = GridSearchCV(
            estimator=RandomForestRegressor(),
            param_grid= param3, 
            cv=10, 
            verbose=-1, 
            return_train_score = True, 
            scoring="neg_mean_squared_error") 

In [24]:
gs_rf3.fit(x_train, y_train)

GridSearchCV(cv=10, estimator=RandomForestRegressor(),
             param_grid={'max_depth': [10, 12, 14],
                         'max_features': [12, 13, 14, 15],
                         'min_samples_leaf': [10, 50, 100],
                         'min_samples_split': [10, 50, 100]},
             return_train_score=True, scoring='neg_mean_squared_error',
             verbose=-1)

In [25]:
bosque4 = gs_rf3.best_estimator_
bosque4

RandomForestRegressor(max_depth=14, max_features=15, min_samples_leaf=10,
                      min_samples_split=10)

In [26]:
y_test4_rf = bosque4.predict(x_test)
y_train4_rf = bosque4.predict(x_train)

In [27]:
rf4_results = fun.metricas(y_test, y_train, y_test4_rf, y_train4_rf, "Random Forest 4")
rf4_results

,MAE,MSE,RMSE,R2,set,modelo
0,0.341919,0.235882,0.485677,0.523668,test,Random Forest 4
1,0.324657,0.235561,0.485346,0.567774,train,Random Forest 4


In [28]:
param4 = {"max_depth": [10, 12, 14], 
        "max_features": [15,16,17,18],
        "min_samples_split": [10, 50, 100],
        "min_samples_leaf": [10,50,100]} 

In [29]:
gs_rf4 = GridSearchCV(
            estimator=RandomForestRegressor(),
            param_grid= param4, 
            cv=10, 
            verbose=-1, 
            return_train_score = True, 
            scoring="neg_mean_squared_error") 

In [30]:
gs_rf4.fit(x_train, y_train)

GridSearchCV(cv=10, estimator=RandomForestRegressor(),
             param_grid={'max_depth': [10, 12, 14],
                         'max_features': [15, 16, 17, 18],
                         'min_samples_leaf': [10, 50, 100],
                         'min_samples_split': [10, 50, 100]},
             return_train_score=True, scoring='neg_mean_squared_error',
             verbose=-1)

In [31]:
bosque5 = gs_rf4.best_estimator_
bosque5

RandomForestRegressor(max_depth=14, max_features=18, min_samples_leaf=10,
                      min_samples_split=10)

In [32]:
y_test5_rf = bosque5.predict(x_test)
y_train5_rf = bosque5.predict(x_train)

In [33]:
rf5_results = fun.metricas(y_test, y_train, y_test5_rf, y_train5_rf, "Random Forest 5")
rf5_results

,MAE,MSE,RMSE,R2,set,modelo
0,0.333794,0.229386,0.478943,0.536785,test,Random Forest 5
1,0.310989,0.220011,0.469054,0.596306,train,Random Forest 5


In [34]:
param5 = {"max_depth": [14, 16, 18], 
        "max_features": [18,19,20,21],
        "min_samples_split": [10, 50, 100],
        "min_samples_leaf": [10,50,100]} 

In [35]:
gs_rf5 = GridSearchCV(
            estimator=RandomForestRegressor(),
            param_grid= param5, 
            cv=10, 
            verbose=-1, 
            return_train_score = True, 
            scoring="neg_mean_squared_error") 

In [36]:
gs_rf5.fit(x_train, y_train)

GridSearchCV(cv=10, estimator=RandomForestRegressor(),
             param_grid={'max_depth': [14, 16, 18],
                         'max_features': [18, 19, 20, 21],
                         'min_samples_leaf': [10, 50, 100],
                         'min_samples_split': [10, 50, 100]},
             return_train_score=True, scoring='neg_mean_squared_error',
             verbose=-1)

In [37]:
bosque6 = gs_rf5.best_estimator_
bosque6

RandomForestRegressor(max_depth=16, max_features=21, min_samples_leaf=10,
                      min_samples_split=10)

In [38]:
y_test6_rf = bosque6.predict(x_test)
y_train6_rf = bosque6.predict(x_train)

In [39]:
rf6_results = fun.metricas(y_test, y_train, y_test6_rf, y_train6_rf, "Random Forest 6")
rf6_results

,MAE,MSE,RMSE,R2,set,modelo
0,0.328614,0.225878,0.475267,0.543869,test,Random Forest 6
1,0.295919,0.208146,0.456230,0.618077,train,Random Forest 6


In [40]:
param6 = {"max_depth": [16, 18, 20], 
        "max_features": [24, 26, 28, 30],
        "min_samples_split": [10, 50, 100],
        "min_samples_leaf": [10,50,100]} 

In [41]:
gs_rf6 = GridSearchCV(
            estimator=RandomForestRegressor(),
            param_grid= param6, 
            cv=10, 
            verbose=-1, 
            return_train_score = True, 
            scoring="neg_mean_squared_error") 

In [42]:
gs_rf6.fit(x_train, y_train)

GridSearchCV(cv=10, estimator=RandomForestRegressor(),
             param_grid={'max_depth': [16, 18, 20],
                         'max_features': [24, 26, 28, 30],
                         'min_samples_leaf': [10, 50, 100],
                         'min_samples_split': [10, 50, 100]},
             return_train_score=True, scoring='neg_mean_squared_error',
             verbose=-1)

In [43]:
bosque7 = gs_rf6.best_estimator_
bosque7

RandomForestRegressor(max_depth=20, max_features=30, min_samples_leaf=10,
                      min_samples_split=10)

In [44]:
y_test7_rf = bosque7.predict(x_test)
y_train7_rf = bosque7.predict(x_train)

In [45]:
rf7_results = fun.metricas(y_test, y_train, y_test7_rf, y_train7_rf, "Random Forest 7")
rf7_results

,MAE,MSE,RMSE,R2,set,modelo
0,0.319841,0.216519,0.465316,0.562769,test,Random Forest 7
1,0.272443,0.185040,0.430163,0.660473,train,Random Forest 7


In [46]:
param7 = {"max_depth": [14, 16, 18], 
        "max_features": [30, 34, 38, 40],
        "min_samples_split": [10, 50, 100],
        "min_samples_leaf": [10,50,100]} 

In [47]:
gs_rf7 = GridSearchCV(
            estimator=RandomForestRegressor(),
            param_grid= param7, 
            cv=10, 
            verbose=-1, 
            return_train_score = True, 
            scoring="neg_mean_squared_error") 

In [48]:
gs_rf7.fit(x_train, y_train)

GridSearchCV(cv=10, estimator=RandomForestRegressor(),
             param_grid={'max_depth': [14, 16, 18],
                         'max_features': [30, 34, 38, 40],
                         'min_samples_leaf': [10, 50, 100],
                         'min_samples_split': [10, 50, 100]},
             return_train_score=True, scoring='neg_mean_squared_error',
             verbose=-1)

In [49]:
bosque8 = gs_rf7.best_estimator_
bosque8

RandomForestRegressor(max_depth=16, max_features=40, min_samples_leaf=10,
                      min_samples_split=10)

In [50]:
y_test8_rf = bosque8.predict(x_test)
y_train8_rf = bosque8.predict(x_train)

In [51]:
rf8_results = fun.metricas(y_test, y_train, y_test8_rf, y_train8_rf, "Random Forest 8")
rf8_results

,MAE,MSE,RMSE,R2,set,modelo
0,0.316545,0.214307,0.462933,0.567236,test,Random Forest 8
1,0.263347,0.175617,0.419067,0.677764,train,Random Forest 8


*Tras varias pruebas con los hiperparámetros nos percatamos de que por el momento el modelo se vuelve más eficinente al aumentar el numero máximo de 'features', mientras que la profundidad máxima se mantiene en torno a los 14-16*

In [52]:
param8 = {"max_depth": [12, 14, 16], 
        "max_features": [40, 50, 60, 70],
        "min_samples_split": [10, 50, 100],
        "min_samples_leaf": [10,50,100]} 

In [53]:
gs_rf8 = GridSearchCV(
            estimator=RandomForestRegressor(),
            param_grid= param8, 
            cv=10, 
            verbose=-1, 
            return_train_score = True, 
            scoring="neg_mean_squared_error") 

In [54]:
gs_rf8.fit(x_train, y_train)

GridSearchCV(cv=10, estimator=RandomForestRegressor(),
             param_grid={'max_depth': [12, 14, 16],
                         'max_features': [40, 50, 60, 70],
                         'min_samples_leaf': [10, 50, 100],
                         'min_samples_split': [10, 50, 100]},
             return_train_score=True, scoring='neg_mean_squared_error',
             verbose=-1)

In [55]:
bosque9 = gs_rf8.best_estimator_
bosque9

RandomForestRegressor(max_depth=16, max_features=70, min_samples_leaf=10,
                      min_samples_split=10)

In [56]:
y_test9_rf = bosque9.predict(x_test)
y_train9_rf = bosque9.predict(x_train)

In [57]:
rf9_results = fun.metricas(y_test, y_train, y_test9_rf, y_train9_rf, "Random Forest 9")
rf9_results

,MAE,MSE,RMSE,R2,set,modelo
0,0.312280,0.210464,0.458763,0.574997,test,Random Forest 9
1,0.247999,0.161423,0.401775,0.703808,train,Random Forest 9


In [58]:
rf_metrics = pd.concat([rf1_results, rf2_results, rf3_results, rf4_results, rf5_results, rf6_results, rf7_results, rf8_results, rf9_results], axis=0)

In [59]:
rf_metrics

,MAE,MSE,RMSE,R2,set,modelo
0,0.295399,0.193418,0.439793,0.609418,test,Random Forest 1
1,0.108429,0.029441,0.171584,0.945979,train,Random Forest 1
0,0.457024,0.355859,0.596539,0.281391,test,Random Forest 2
1,0.464539,0.397166,0.630211,0.271249,train,Random Forest 2
0,0.384101,0.273459,0.522933,0.447787,test,Random Forest 3
1,0.384721,0.298979,0.546790,0.451409,train,Random Forest 3
0,0.341919,0.235882,0.485677,0.523668,test,Random Forest 4
1,0.324657,0.235561,0.485346,0.567774,train,Random Forest 4
0,0.333794,0.229386,0.478943,0.536785,test,Random Forest 5
1,0.310989,0.220011,0.469054,0.596306,train,Random Forest 5


In [60]:
# calculamos importancia de cada una de las variables incluidas en los dos modelos que mejor han funcionado: bosque y bosque9
importancia_predictores1 = pd.DataFrame(
                            {'predictor': x_train.columns,
                             'importancia': bosque.feature_importances_,
                             'modelo':'modelo 1'}
                            )


# ordenamos de mayor a menor los resultados
importancia_predictores1.sort_values(by=["importancia"], ascending=False, inplace = True)

# printeamos los resultados
print("Importancia de los predictores en el modelo")
print("-------------------------------------------")
importancia_predictores1

Importancia de los predictores en el modelo
-------------------------------------------


,predictor,importancia,modelo
12,public_transport_ticket,0.184600,modelo 1
11,cigarettes_marlboro,0.149484,modelo 1
22,country_pesos,0.096528,modelo 1
17,cinema,0.052129,modelo 1
14,gasoline,0.051630,modelo 1
...,...,...,...
27,country_Angola,0.000000,modelo 1
50,country_British Virgin Islands,0.000000,modelo 1
88,country_French Polynesia,0.000000,modelo 1
101,country_Guyana,0.000000,modelo 1


In [61]:
importancia_predictores9 = pd.DataFrame(
                            {'predictor': x_train.columns,
                             'importancia': bosque.feature_importances_,
                             'modelo':'modelo 9'}
                            )


# ordenamos de mayor a menor los resultados
importancia_predictores9.sort_values(by=["importancia"], ascending=False, inplace = True)

# printeamos los resultados
print("Importancia de los predictores en el modelo")
print("-------------------------------------------")
importancia_predictores9

Importancia de los predictores en el modelo
-------------------------------------------


,predictor,importancia,modelo
12,public_transport_ticket,0.184600,modelo 9
11,cigarettes_marlboro,0.149484,modelo 9
22,country_pesos,0.096528,modelo 9
17,cinema,0.052129,modelo 9
14,gasoline,0.051630,modelo 9
...,...,...,...
27,country_Angola,0.000000,modelo 9
50,country_British Virgin Islands,0.000000,modelo 9
88,country_French Polynesia,0.000000,modelo 9
101,country_Guyana,0.000000,modelo 9


In [62]:
comp_import = pd.concat([importancia_predictores1, importancia_predictores9], axis = 0)

In [63]:
comp_import

,predictor,importancia,modelo
12,public_transport_ticket,0.184600,modelo 1
11,cigarettes_marlboro,0.149484,modelo 1
22,country_pesos,0.096528,modelo 1
17,cinema,0.052129,modelo 1
14,gasoline,0.051630,modelo 1
...,...,...,...
27,country_Angola,0.000000,modelo 9
50,country_British Virgin Islands,0.000000,modelo 9
88,country_French Polynesia,0.000000,modelo 9
101,country_Guyana,0.000000,modelo 9


*Comprobamos que la variable de pais no está aportando información al modelo predictivo, si bien ninguna de nuestras variables predictoras tiene mucho peso lo que podría ser parte de la explicación de que ninguno de nuestros modelos alcance un elevado nivel de eficiencia*

- Debatid entre vosotras que modelo es mejor y por qué (basándose en las métricas)

*Para tomar esta decisión primero creamos un dataframe con las métricas de los distintos modelos generados*

In [65]:
reg_lin = pd.read_csv('../archivos/metricas_reg_lineal.csv', index_col=0)

In [66]:
dec_tree = pd.read_csv('../archivos/metricas_decision_tree.csv', index_col=0)
dec_tree

,MAE,MSE,RMSE,R2,set,modelo
0,4.424041e-01,4.866278e-01,6.975872e-01,0.014296,test,Decission Tree Label
1,2.130265e-19,1.478168e-35,3.844695e-18,1.000000,train,Decission Tree Label
0,4.085831e-01,3.699387e-01,6.082259e-01,0.252958,test,Decission Tree Dummie 1
1,1.922077e-19,4.142582e-35,6.436289e-18,1.000000,train,Decission Tree Dummie 1
0,4.294585e-01,3.471063e-01,5.891573e-01,0.299065,test,Decission Tree 2 Dummie
1,4.368916e-01,3.853910e-01,6.207987e-01,0.292854,train,Decission Tree 2 Dummie
0,4.323555e-01,4.719384e-01,6.869777e-01,0.044050,test,Decission Tree Orden 1
1,1.331416e-19,1.256443e-35,3.544634e-18,1.000000,train,Decission Tree Orden 1


In [91]:
total_metrics = pd.concat([reg_lin,dec_tree, rf1_results, rf9_results, rfor_results,rfor2_results ], axis=0)

In [92]:
total_metrics

,MAE,MSE,RMSE,R2,set,modelo
0,3.590157e-01,2.088367e-01,4.569866e-01,5.402636e-01,test,Linear Regresion Label 1
1,3.716074e-01,2.241389e-01,4.734331e-01,5.272659e-01,train,Linear Regression Label 1
0,5.113565e+06,3.526986e+15,5.938843e+07,-7.764362e+15,test,Linear Regresion Dummie 1
1,2.422486e-01,1.075410e-01,3.279345e-01,7.731839e-01,train,Linear Regression Dummie 1
0,3.285266e-01,1.769487e-01,4.206528e-01,6.104623e-01,test,Linear Regresion Orden 1
1,3.313081e-01,1.788406e-01,4.228955e-01,6.228051e-01,train,Linear Regression Orden 1
0,4.424041e-01,4.866278e-01,6.975872e-01,1.429561e-02,test,Decission Tree Label
1,2.130265e-19,1.478168e-35,3.844695e-18,1.000000e+00,train,Decission Tree Label
0,4.085831e-01,3.699387e-01,6.082259e-01,2.529584e-01,test,Decission Tree Dummie 1
1,1.922077e-19,4.142582e-35,6.436289e-18,1.000000e+00,train,Decission Tree Dummie 1


*Podemos concluir que las que los modelos con mayor capacidad predictiva son los realizados con el método Random Forest. No siendo tan clara la diferencia entre ellos, si bien observamos un a menor diferencia entre los valores R2 en el modelo Random Forest Orden 2. En caso de disponer de más tiempo, profundizaríamos en la mejora de Random Forest 2, en el que se percibe una menor distancia y mejores resultados con menor overfitting.*



In [1]:
total_metrics.to_csv('../archivos/total_metrics_reg_lineal.csv')

NameError: name 'total_metrics' is not defined